### Generating the dataset with sentences of text extracted from the REGBR database
#### Author: Mauricio Barros 
#### 2022-08-11

In [7]:
import pandas as pd
import os
from pathlib import Path
from src.mongodbclient import MongoDbClient
from src.cfgparser import getMongoDbStrConn
from src.preprocessing import PreProcessing
from src.sentenceregbr import SentenceRegBr

In [8]:
BASE_DIR = Path(os.path.abspath('')).resolve()

In [3]:
objSentecesRegBr = SentenceRegBr()

  Abbreviation: [1.7426] art
  Abbreviation: [1.9803] art


In [9]:
mongoClient = MongoDbClient(getMongoDbStrConn())
mongoRegBrCollection = mongoClient.getCollection('regbr','regbr_full_db')
mongoRegBrSentenceCollection = mongoClient.getCollection('regbr','regbr_sentences')

In [5]:
cursor = mongoRegBrCollection.find( {},{"id":1,"texto_lei":1})
for document in cursor:
    try:
        idDoc = document["id"]
        sentenceList = objSentecesRegBr.getSentencesRegBrFormat(document["texto_lei"],idDoc,-1)
        for sentenca in sentenceList:
            mongoRegBrSentenceCollection.insert_one(sentenca)
    except Exception as e:
        print(f'Erro ao inserir id: {idDoc}. Detalhe: {e}')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mbjesus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mbjesus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
cursor = mongoRegBrSentenceCollection.find( { },{"id_sentenca":1, "id_doc_base":1,"tokens_total":1,"tokens_no_stopwords":1,"text_sentenca":1})
dfSentenca =  pd.DataFrame(list(cursor))

In [14]:
dfSentenca.drop(["_id"],inplace=True, axis=1)
dfSentenca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693 entries, 0 to 1692
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_sentenca          1693 non-null   object
 1   id_doc_base          1693 non-null   int64 
 2   tokens_total         1693 non-null   int64 
 3   tokens_no_stopwords  1693 non-null   int64 
 4   text_sentenca        1693 non-null   object
dtypes: int64(3), object(2)
memory usage: 66.3+ KB


In [11]:
dfSentenca.to_parquet(Path(os.path.join(BASE_DIR,'dataset', 'regbr','dados_regbr_sentencas.parquet')), engine='fastparquet')

In [10]:
cursor = mongoRegBrSentenceCollection.find( { },{"id_sentenca":1, "ordem":1,"id_doc_base":1,"tokens_total":1,"tokens_no_stopwords":1,"cat_sentenca":1,"text_sentenca":1 })
dfSentenca =  pd.DataFrame(list(cursor))

In [11]:
dfSentenca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883635 entries, 0 to 883634
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   _id                  883635 non-null  object
 1   id_sentenca          883635 non-null  object
 2   ordem                883635 non-null  int64 
 3   id_doc_base          883635 non-null  int64 
 4   tokens_total         883635 non-null  int64 
 5   tokens_no_stopwords  883635 non-null  int64 
 6   cat_sentenca         883635 non-null  int64 
 7   text_sentenca        883635 non-null  object
dtypes: int64(5), object(3)
memory usage: 53.9+ MB


In [12]:
dfSentenca.drop(["_id"],inplace=True, axis=1)

In [13]:
dfSentenca.to_parquet(Path(os.path.join(BASE_DIR,'dataset', 'regbr','dados_regbr_sentencas.parquet')), engine='fastparquet')